In [1]:
!pip3 install pymongo

In [10]:
from pymongo import MongoClient

dbName = "datamad0320"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()

mongodb://localhost/datamad0320


In [16]:
cur = db.list_collections()

In [17]:
for collection in cur:
    print(collection)

{'name': 'companies', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': UUID('7e184b68-86d8-44c9-ba2c-1a431488941f')}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'ns': 'datamad0320.companies'}}


# Making queries

In [25]:
cursor = db.companies.find({"$or":[{"founded_year":2004},{"founded_year":2005}]})
for i,company in enumerate(cursor):
    if i >= 2:
        break
    print(company["name"])


Digg
Zoho


In [27]:
query = {"$or":[{"founded_year":2004},{"founded_year":2005}]}
for c in db.companies.find(query).limit(3):
    print(c["name"])

Digg
Zoho
Wetpaint


In [32]:
def byNextNYears(yearStart,numYears=2):
    years = list(range(yearStart,yearStart+numYears))
    query = {"$or":[{"founded_year":year} for year in years]}
    return query

byNextNYears(2004)

{'$or': [{'founded_year': 2004}, {'founded_year': 2005}]}

In [33]:
for c in db.companies.find(byNextNYears(2004)).limit(3):
    print(c["name"])

Digg
Zoho
Wetpaint


In [39]:
import re
searchre = re.compile("^Face.*")
cur = db.companies.find({"name":searchre})
for c in cur:
    print(c["name"])

Facebook
FaceTec
Face Your Manga
Facebookster
Facebook Causes Application
FaceKoo
FacebookLicious!
FaceTime Strategy


In [41]:
list(db.companies.find({"name":searchre},{"name":1}))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8e'), 'name': 'Facebook'},
 {'_id': ObjectId('52cdef7c4bab8bd675298334'), 'name': 'FaceTec'},
 {'_id': ObjectId('52cdef7d4bab8bd675299497'), 'name': 'Face Your Manga'},
 {'_id': ObjectId('52cdef7d4bab8bd675299d49'), 'name': 'Facebookster'},
 {'_id': ObjectId('52cdef7d4bab8bd67529a054'),
  'name': 'Facebook Causes Application'},
 {'_id': ObjectId('52cdef7d4bab8bd67529a1b9'), 'name': 'FaceKoo'},
 {'_id': ObjectId('52cdef7f4bab8bd67529c4a8'), 'name': 'FacebookLicious!'},
 {'_id': ObjectId('52cdef7f4bab8bd67529c661'), 'name': 'FaceTime Strategy'}]

In [43]:
list(map( lambda doc: doc["name"] ,db.companies.find({"name":searchre},{"name":1,"_id":0})))

['Facebook',
 'FaceTec',
 'Face Your Manga',
 'Facebookster',
 'Facebook Causes Application',
 'FaceKoo',
 'FacebookLicious!',
 'FaceTime Strategy']

In [47]:
list(db.companies.find({"name":searchre},{"name":1}))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8e'), 'name': 'Facebook'},
 {'_id': ObjectId('52cdef7c4bab8bd675298334'), 'name': 'FaceTec'},
 {'_id': ObjectId('52cdef7d4bab8bd675299497'), 'name': 'Face Your Manga'},
 {'_id': ObjectId('52cdef7d4bab8bd675299d49'), 'name': 'Facebookster'},
 {'_id': ObjectId('52cdef7d4bab8bd67529a054'),
  'name': 'Facebook Causes Application'},
 {'_id': ObjectId('52cdef7d4bab8bd67529a1b9'), 'name': 'FaceKoo'},
 {'_id': ObjectId('52cdef7f4bab8bd67529c4a8'), 'name': 'FacebookLicious!'},
 {'_id': ObjectId('52cdef7f4bab8bd67529c661'), 'name': 'FaceTime Strategy'}]

In [49]:
list(db.companies.find({"name":searchre},{"name":1}).skip(2).limit(3))

[{'_id': ObjectId('52cdef7d4bab8bd675299497'), 'name': 'Face Your Manga'},
 {'_id': ObjectId('52cdef7d4bab8bd675299d49'), 'name': 'Facebookster'},
 {'_id': ObjectId('52cdef7d4bab8bd67529a054'),
  'name': 'Facebook Causes Application'}]

In [51]:
db.companies.find_one({"name":searchre},{"name":1})

{'_id': ObjectId('52cdef7c4bab8bd675297d8e'), 'name': 'Facebook'}

In [56]:
%time
len(list(db.companies.find({"founded_year":2004},{"name":1})))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 561 µs


752

In [57]:
%time
db.companies.find({"founded_year":2004},{"name":1}).count()

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 7.15 µs


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


752

In [67]:
from pymongo import ASCENDING, DESCENDING
list(db.companies.find({"founded_year":2004},{"name":1}).sort([
   ('name', DESCENDING)
]).limit(5))

[{'_id': ObjectId('52cdef7d4bab8bd675298c34'), 'name': 'zebraspot design'},
 {'_id': ObjectId('52cdef7c4bab8bd6752981dd'), 'name': 'yoonew'},
 {'_id': ObjectId('52cdef7e4bab8bd67529a985'), 'name': 'webwork'},
 {'_id': ObjectId('52cdef7e4bab8bd67529bc37'), 'name': 't3leads'},
 {'_id': ObjectId('52cdef7c4bab8bd675297e2e'), 'name': 'stylediary'}]

In [87]:
data = list(db.companies.find({"founded_year":2004},{"name":1,"funding_rounds.raised_amount":1}))

In [88]:
import pandas as pd
df = pd.DataFrame(data).explode("funding_rounds")

In [94]:
df.funding_rounds

0       {'raised_amount': 8500000}
0       {'raised_amount': 2800000}
0      {'raised_amount': 28700000}
0       {'raised_amount': 5000000}
1        {'raised_amount': 500000}
                  ...             
747                            NaN
748                            NaN
749                            NaN
750     {'raised_amount': 2000000}
751                            NaN
Name: funding_rounds, Length: 1363, dtype: object

In [96]:
df.funding_rounds = df.funding_rounds.apply(lambda doc: doc.get("raised_amount") if type(doc)==dict else 0)

In [100]:
df.describe()

,funding_rounds
count,1.309000e+03
mean,1.093028e+07
std,4.792119e+07
min,0.000000e+00
25%,0.000000e+00
50%,3.900000e+06
75%,1.100000e+07
max,1.500000e+09


In [103]:
rests = list(db.restaurantes.find())

In [107]:
from pandas.io.json import json_normalize
pd.DataFrame(json_normalize(rests))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,_id,borough,cuisine,grades,name,restaurant_id,address.building,address.coord,address.street,address.zipcode,visitado,stars
0,5e983c0b4b5624849fa69583,Manhattan,Irish,"[{'date': 2014-09-06 00:00:00, 'grade': 'A', '...",Dj Reynolds Pub And Restaurant,30191841,351,"[-73.98513559999999, 40.7676919]",West 57 Street,10019,NaN,NaN
1,5e983c0b4b5624849fa69584,Queens,Jewish/Kosher,"[{'date': 2014-11-24 00:00:00, 'grade': 'Z', '...",Tov Kosher Kitchen,40356068,97-22,"[-73.8601152, 40.7311739]",63 Road,11374,NaN,NaN
2,5e983c0b4b5624849fa69585,Brooklyn,Hamburgers,"[{'date': 2014-12-30 00:00:00, 'grade': 'A', '...",Wendy'S,30112340,469,"[-73.961704, 40.662942]",Flatbush Avenue,11225,NaN,NaN
3,5e983c0b4b5624849fa69586,Brooklyn,American,"[{'date': 2014-06-10 00:00:00, 'grade': 'A', '...",Riviera Caterer,40356018,2780,"[-73.98241999999999, 40.579505]",Stillwell Avenue,11224,NaN,NaN
4,5e983c0b4b5624849fa69587,Staten Island,Jewish/Kosher,"[{'date': 2014-10-06 00:00:00, 'grade': 'A', '...",Kosher Island,40356442,2206,"[-74.1377286, 40.6119572]",Victory Boulevard,10314,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
25356,5e983c0d4b5624849fa6f88f,Brooklyn,Other,[],Cold Press'D,50018995,921,"[-73.9691347, 40.6389857]",Cortelyou Rd,11218,NaN,NaN
25357,5e983c0d4b5624849fa6f890,NaN,NaN,NaN,pepe rest,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25358,5e983c0d4b5624849fa6f891,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
25359,5e983c0d4b5624849fa6f892,NaN,NaN,NaN,La tratoria de Giorgio,NaN,NaN,NaN,NaN,NaN,NaN,5.0
